<a href="https://colab.research.google.com/github/napoles-uach/DGL_Smell/blob/main/dgl_smell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dgl

     |████████████████████████████████| 4.4 MB 5.1 MB/s 


In [2]:
pip install ogb

     |████████████████████████████████| 67 kB 2.9 MB/s 
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=ef5f602054c7d4039d7e056c69a1c0fa90c0ffa8cc7c7b0515cc148f931c493a
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built littleutils


In [3]:
pip install rdkit-pypi

     |████████████████████████████████| 18.6 MB 154 kB/s 


In [4]:
!wget https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv

--2021-08-05 20:10:11--  https://raw.githubusercontent.com/napoles-uach/DGL_Smell/main/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 197190 (193K) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>] 192.57K  --.-KB/s    in 0.03s   

2021-08-05 20:10:11 (6.35 MB/s) - ‘train.csv’ saved [197190/197190]



In [5]:
import pandas as pd
df=pd.read_csv('train.csv')
df.head()

,SMILES,SENTENCE
0,C/C=C/C(=O)C1CCC(C=C1C)(C)C,"fruity,rose"
1,COC(=O)OC,"fresh,ethereal,fruity"
2,Cc1cc2c([nH]1)cccc2,"resinous,animalic"
3,C1CCCCCCCC(=O)CCCCCCC1,"powdery,musk,animalic"
4,CC(CC(=O)OC1CC2C(C1(C)CC2)(C)C)C,"coniferous,camphor,fruity"


In [6]:
import dgl
from dgl.data import DGLDataset
import torch
import torch as th
import os
from ogb.utils.features import (allowable_features, atom_to_feature_vector,
 bond_to_feature_vector, atom_feature_vector_to_dict, bond_feature_vector_to_dict) 
from rdkit import Chem
import numpy as np

Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


In [7]:
def smiles2graph(smiles_string):
    """
    Converts SMILES string to graph Data object
    :input: SMILES string (str)
    :return: graph object
    """

    mol = Chem.MolFromSmiles(smiles_string)

    A = Chem.GetAdjacencyMatrix(mol)
    A = np.asmatrix(A)
    nnodes=len(A)
    nz = np.nonzero(A)
    edge_list=[]
    src=[]
    dst=[]

    for i in range(nz[0].shape[0]):
      src.append(nz[0][i])
      dst.append(nz[1][i])

    u, v = src, dst
    g = dgl.graph((u, v))
    bg=dgl.to_bidirected(g)

    return bg

In [8]:
def feat_vec(smiles_string):
    """
    Returns atom features for a molecule given a smiles string
    """
    # atoms
    mol = Chem.MolFromSmiles(smiles_string)
    atom_features_list = []
    for atom in mol.GetAtoms():
        atom_features_list.append(atom_to_feature_vector(atom))
    x = np.array(atom_features_list, dtype = np.int64)
    return x

In [9]:
# this block uses the column SENTENCE to build one hot encoding for each sentence
# All sentences are stored in a list, repetitions are eliminated and ordered alphabetically
# this gives 109 strings whose indexes are used to build one hot encoding vectors on 109 entries.
# Example: fresh,ethereal,fruity gives a vector with ones in entries 42, 47, and 48.
lista_senten=df['SENTENCE'].to_list()
olores=[]
for olor in lista_senten:
  olor=olor.split(",")
  olores.append(olor)
# da formato correcto a las sentencias en forma de lista
from pandas.core.common import flatten
olores=list(flatten(olores))
#crea una sola lista con los olores
olores = list(dict.fromkeys(olores))
#elimina repeticiones
olores.sort()
#ordena alfabeticamente
inoh=[]
for olor in lista_senten:
  olor=olor.split(",")
  indices=[]
  for t in olor:
    if t in olores:
      indices.append(olores.index(t))
  inoh.append(indices)

onehot=[]
for i in inoh:
  ohv=np.zeros(len(olores))
  ohv[i]=1
  onehot.append(ohv)

In [10]:
lista_senten[1]

'fresh,ethereal,fruity'

In [11]:
onehot[1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [13]:
# This block makes a list of graphs
lista_mols=df['SMILES'].to_list()

j=0
graphs=[]
execptions=[]
for mol in lista_mols:
  
  g_mol=smiles2graph(mol)

  try:
    g_mol.ndata['feat']=torch.tensor(feat_vec(mol)) 
  except:
    execptions.append(j)
   

  graphs.append(g_mol) #agrega grafo de mol
  j+=1

In [14]:
labels=onehot

In [15]:
# Some smiles are not well processed, so they are droped
for i in execptions:
  graphs.pop(i)
  labels.pop(i)

In [16]:
class SyntheticDataset(DGLDataset):
    def __init__(self):
        super().__init__(name='synthetic')

    def process(self):
        #edges = pd.read_csv('./graph_edges.csv')
        #properties = pd.read_csv('./graph_properties.csv')
        self.graphs = graphs
        self.labels = torch.LongTensor(labels)

    def __getitem__(self, i):
        return self.graphs[i], self.labels[i]

    def __len__(self):
        return len(self.graphs)

dataset = SyntheticDataset()
graph, label = dataset[0]
print(graph, label)

Graph(num_nodes=14, num_edges=28,
      ndata_schemes={'feat': Scheme(shape=(9,), dtype=torch.int64)}
      edata_schemes={}) tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
